# Updated single qubit operations 

In [ ]:
# Import all necessary packages

# here are the objects that we use to create a quantum circuit in qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.visualization import plot_bloch_multivector, plot_bloch_vector, plot_histogram
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace
from qiskit import QuantumCircuit , transpile
from qiskit.visualization import plot_state_city, plot_state_hinton, plot_state_qsphere

# Used for display, plotting and making gifs in the jupyter environment 
from IPython.display import Markdown
from PIL import Image
from IPython.display import Image as IPImage
import io


## 2. Single Qubit gates and transformations 
### The Pauli Gates (X, Y & Z Gates)
We will now learn about some common quantum gates that will be used to transform our states.

The Pauli X, Y, and Z gates are operations that rotate a single qubit $180^\circ$ ($\pi$ radians) around the corresponding X, Y, or Z axis of the Bloch sphere.

For now we will consider how these gates impact the state $\ket{0}$, the initial value of every qubit in `qiskit`.

#### The X-Gate (Quantum `not`)
The Pauli X-gate corresponds to a rotation around the x-axis. This means it will flip state $\ket{0}$ to state $\ket{1}$ and visa versa making it analogous to a classical `not` operation.

#### The Y gate
The Y-gate does a $\pi$ radian rotation about the Y-axis

#### The Z-Gate
The Z-gate performs a $\pi$ radian rotation around the Z-axis.

Lets apply the different rotation gates to the states below:

In [ ]:
# Create a quantum circuit object with three qubits 
QuCirc = QuantumCircuit(3)


# Apply a Z-gate to all three qubits
# Note the shorthand: range(3) = [0,1,2] and range(n) = [0, ..., n]
QuCirc.x(0) # Apply an X-gate to qubit 0
QuCirc.y(1) # Apply a Y-gate to qubit 1
QuCirc.z(2) # Apply a Z-gate to qubit 2

# Display a plot of the circuit using matplotlib (using coloured blocks) and show the initial state (all qubits |0>)
display(QuCirc.draw('mpl', initial_state=True))

# Display the state after Z-gates were applied
display(plot_bloch_multivector(QuCirc))

#### Investigation:
- Why is qubit 2 unaffected by the Z-gate while the fist to qubits were flipped by the X and Y gates?
- Test you hypothesis by applying a gate to every qubit in the circuit without changing the initial state 

In [ ]:
# Create a quantum circuit object with three qubits 
QuCirc = QuantumCircuit(3)

# Initial qubits as follows q_0=|0>   q_1=|+> and q_2=|i> (aka 'right')
# Note: We use `[::-1]` to reverse the string order because by default `initialize` will take the MSB first.
QuCirc.initialize('0+r'[::-1], [0,1,2])     
# Record the initial state (before we gates) for reference
initial_state = Statevector(QuCirc)
# -------------------------------------------------
# <Apply your gates here>>

# -------------------------------------------------
# Display a plot of the circuit
display(QuCirc.draw('mpl'), initial_state=True)

# Display the state BEFORE gates were applied
display(plot_bloch_multivector(QuCirc, title="Before Gates"))
# Display the state AFTER gates
display(plot_bloch_multivector(QuCirc, title="After Gates"))

Let's initialize some common qubit states and see how the X, Y and Z gates effect them.

#### Challenge:
- Can you use X, Y and Z gates to invert all six initial states?
That is transform $\ket{0}\leftrightarrow\ket{1}$, $\ket{+}\leftrightarrow\ket{-}$ and $\ket{i}\leftrightarrow\ket{-i}$ (these are `r` and `l` in qiskit).
- **Bonus** Instead try and do the inversion by applying at least one of each gate (X, Y and Z) to every qubit. What is the the smallest number of gates needed?


In [ ]:
# Make circuit
QuCirc = QuantumCircuit(6)

# Initialize qubit states 
QuCirc.initialize('01+-rl'[::-1],range(6)) # Play around with `01+-rl` to change the initial states
# Note: We use `[::-1]` to reverse the string order because by default `initialize` will take the MSB (qubit 6) first.

# Add a barrier (dotted line) to separate initiation and applying gates
QuCirc.barrier()

# Store initial state vector for reference 
initial_state = Statevector(QuCirc)
# -------------------------------------------------

# <Your gates go here...>

# -------------------------------------------------

# Display qubit states on bloch sphere
display(plot_bloch_multivector(initial_state, title="Initial State")) # Before gates were applied
display(plot_bloch_multivector(QuCirc, title="Final State")) # After gates 

# Display Circuit
display(QuCirc.draw('mpl'))

### Mathematic Representation  
Recall that we describe qubits as a superposition of two states:
$$
\ket{\psi}=\alpha\ket{0}+\beta\ket{1}=\alpha
\begin{bmatrix}
    1\\0
\end{bmatrix}
+\beta
\begin{bmatrix}
    0\\1
\end{bmatrix}
=\begin{bmatrix}
    \alpha\\\beta
\end{bmatrix}
$$
Coefficients $\alpha$ and $\beta$ are complex values where $|\alpha|^2=\alpha\alpha^*$ and $|\beta|^2=\beta\beta^*$ are the probabilities of measuring `0` and `1` respectively.

#### Applying a Quantum Gate
We can then describe single qubit operation (gate) as a $2\times2$ complex matrix operator $U$ and apply it through multiplication:
$$
\ket{\psi'}=
\begin{bmatrix}
    \alpha'\\\beta'
\end{bmatrix}
= \begin{bmatrix}
    U_{0,0} & U_{0,1} \\
    U_{1,0} & U_{1,1}
\end{bmatrix}
\begin{bmatrix}
    \alpha\\\beta
\end{bmatrix}
=U\ket{\psi}
$$

**Note:** The indexing used for matrix values here matches how you would access them in `python`. For example $U_{1,0}$ is `u[1,0]`

The matrix operator $U$ is valid if, and only if, **probity is conserved**. This means that $U$ must be a unitary matrix such that 
$$
U^\dagger U = UU^\dagger=\mathbb{I}
$$
were $U^\dagger$ denotes the Hermitian Adjoint (complex conjugate) and $\mathbb{I}$ is the identity matrix.

#### X, Y and Z matrices  
The matrices for the Pauli gates are as follows:
$$
X=\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}
\hspace{15pt}
Y=\begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix}
\hspace{15pt}
Z=\begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}
$$


#### Fiddle
- See how applying a the Pauli gates works algebraically 

In [ ]:
# These functions will help us visualize matrices in jupyter

from typing import Literal, Iterable # Used to annotate functions (no very important)

def fmt_complex(z:complex, fmt_str:str = "{:.2g}"):
    """
    Formats the given complex number as a sting without displaying superfluous zeros.
        Args:
        - `fmt_str`:  A python style sting format used to covert numbers to letters.
            Default is `"{:.2g}"` which shows upt to two decimal places if they exist.
    """

    if z.imag == 0: # Only plot real if imaginary component is 0
        return fmt_str.format(z.real)
    if z.real == 0: # # Only plot imaginary if imaginary component is 0 (We know imag is non-zero)
        return fmt_str.format(z.imag)+'j'
    
    # No special case - plot both real and imag
    return fmt_str.format(z)

def mat2latex(matrix:Iterable|Iterable[Iterable], matrix_type:Literal['b', 'p', '']='b', fmt_str:str = "{:.2g}")->str:
    """
    Formats the given input `matrix` as a latex equation.

    Args:
        * `matrix_type` defines the brackets used, [`'b'`:[ ], `'p'`:( ), `''`:no brackets]
        * `fmt_str` is a python string format that is used to prase matrix entries as strings
    """
    matrix_str = f"\\begin{{{matrix_type}matrix}}\n" # Latex style matrix begin

    # Loop through all matrix rows
    for row in matrix:
        if isinstance(row, Iterable): # If row is a list we should loop through it

            # Special case where row is a list/array but only has one element 
            if isinstance(row, np.ndarray) and row.size == 1:
                matrix_str += fmt_complex(row.item(), fmt_str) + ' \\\\ \n'
            else:
                # Join all elements in row using `&` as per latex format
                matrix_str += " & ".join([fmt_complex(v, fmt_str) for v in row]) + '\\\\ \n'
        else:
            # Row is a single value, convert it to string
            matrix_str += fmt_complex(row, fmt_str) + ' \\\\ \n'
    matrix_str += f"\\end{{{matrix_type}matrix}}" # Latex style matrix end
    
    return matrix_str

In [ ]:
from qiskit.circuit.library import XGate, YGate, ZGate # These will let us get quantum gates as matrices

# Define our initial state, in this case |0> (You can change this and see what happens)
initial_state = np.matrix([
    [1],
    [0]
], dtype=np.complex128)

# Ensure that input state is valid probability distribution by normalizing it
initial_state /= np.sqrt(np.sum(np.square(initial_state),axis=0))

# Example: Create an X gate matrix (Change this to anything you like)
operation = XGate().to_matrix()

# Apply the operation
final_state = operation * initial_state

# Display the matrices and markdown
display(Markdown(f"$${mat2latex(operation)}{mat2latex(initial_state)}={mat2latex(final_state)}$$"))

# Display bloch shperes for initial and final states
display(plot_bloch_multivector(Statevector(initial_state), title="Initial State"))
display(plot_bloch_multivector(Statevector(final_state), title="Final State"))

#### Question:
- Quantum gates *must* be unitary operation. What does this imply about their physical and computational properties?

#### Challenge:
- Edit the code above to include the inverse operation of each gate to show that the final state is the same as the initial.
The `.conj()` and `.transpose()` might be helpful.

## The Hadamard (H) Gate - Where the magic happens 

The power of quantum computing is in superposition

And to make superposition we use H-gates!

### Algebraic Description
The operator for this gate is as follows:
$$
H=\frac{1}{\sqrt{2}}\begin{bmatrix}1&1\\1&-1\end{bmatrix}
$$

Note that this matrix is not purely diagonal which meas it will translate from the deterministic states $\ket{0}$ and $\ket{1}$ to the probabilistic states $\ket{+}$ and $\ket{-}$

Lets see how that works in code:

In [ ]:
QuCirc = QuantumCircuit(1)

QuCirc.h(0) # Apply a H-Gate to qubit 0

display(QuCirc.draw('mpl', initial_state=True))
display(plot_bloch_multivector(QuCirc))

The power to Hadamard gates is that they create a superposition so we can make it so

In [ ]:
QuCirc = QuantumCircuit(2)

QuCirc.initialize('01'[::-1], [0,1]) # Set qubit 0 = |0> and qubit |1>

QuCirc.h([0,1]) # Apply a H-Gate to both qubits

QuCirc.measure_all() # Measure all qubit values

display(QuCirc.draw('mpl', initial_state=True))

# Make an AER simulator object
aer_sim = AerSimulator()
# Transpile the quantum circuit to run with the simulator
tqc = transpile(QuCirc, aer_sim)

# Run the simulation
result = aer_sim.run(tqc, shots=1024).result()
# Get the counts (number of measured values) from the circuit
counts = result.get_counts()

# Print the raw counts to the console
print("Measurement results:", counts)

# Plot the measurement results as a histogram for visualization
display(plot_histogram(counts))

#### Investigation:  
- How does changing the number of shots impact the distribution?
- What happens if you remove on of the H-gates?

Now lets apply the H-gate effects some more common qubit states:

In [ ]:
QuCirc = QuantumCircuit(6)
QuCirc.initialize('01+-lr'[::-1], range(6))
initial_state = Statevector(QuCirc)

QuCirc.h(range(6))

final_state = Statevector(QuCirc)

display(plot_bloch_multivector(Statevector(initial_state), title="Initial State"))
display(plot_bloch_multivector(Statevector(final_state), title="After H-Gate"))

#### Questions
- What is the inverse of the H-Gate?
- Is the H-Gate also a rotation?

### Arbitrary Rotations on any axis 
We can make arbitrary gates with even smaller rogations by following the same logic as the Pauli gates.
For example a $\frac\pi3$ radian ration would use the third primary root of unity of the respective Pauli gate for the axis of rotation.

So $\sqrt[3]{Z}$ is a $\frac \pi 3$ rotation around the Z-axis and more generally: $\sqrt[n]{Z}$ is a $\frac \pi n$ rotation.
The same would apply for X anf Y gates.

Based on this we can define rotations as parametric gates as follows:
$$
R_x(\theta)=\begin{bmatrix}
\cos(\frac{\theta}{2})&-i\sin(\frac{\theta}{2})\\
-i\sin(\frac{\theta}{2})&\cos(\frac{\theta}{2})
\end{bmatrix}
\hspace{15pt}
R_z(\theta)=\begin{bmatrix}
\cos(\frac{\theta}{2})&-\sin(\frac{\theta}{2})\\
\sin(\frac{\theta}{2})&\cos(\frac{\theta}{2})
\end{bmatrix}
\hspace{15pt}
R_z(\theta)=\begin{bmatrix}
e^{-i\frac{\theta}{2}}&0\\
0&e^{i\frac{\theta}{2}}
\end{bmatrix}
$$


In `qiskit` we can apply these gates using parameterized `rz`, `rx` and `ry` gates as well.

Try some combinations out below, mess about and have fun!

In [ ]:
QuCirc = QuantumCircuit(3)
QuCirc.rz(np.pi/3,0) #You have to specify the rotation angle and the target qubit in that order 
QuCirc.ry(np.pi/3,1) # For example this: `ry(np.pi/3,1)` rotates qubit 1 π/3 rad around the y axis.
QuCirc.rx(np.pi/3,2) 
display(plot_bloch_multivector(QuCirc))
display(QuCirc.draw('mpl', initial_state=True))

#### Math Challenge
- Can you get the matrix definitions for the X, Y and Z gates from the $R_x$, $R_y$ and $R_z$?
- Why is it okay to ignore complex exponential factors (phases) if they apply to the whole gate?


#### Investigation
- We now know how to move from probabilistic to deterministic states using quatre X and Y rotations. Is the H-gate still special?
- Why is The H-gate different? Can you find something it can do that the others can't?
- Why is this property of the H-gate useful?


#### Challenge
- The S-Gate is defined as follows:
$$
S=\begin{bmatrix}
1&0\\
0&i
\end{bmatrix}
$$
- Can you show mathematically and/or by messing around with the code above that this is a special case of one of the rotation gates?

<details>
<summary><b>Hint</b></summary>

Global phase is irrelevant so
$$\begin{bmatrix}
e^{-i\frac{\theta}{2}}&0\\
0&e^{i\frac{\theta}{2}}
\end{bmatrix}
\equiv
\begin{bmatrix}
1&0\\
0&e^{i\theta}
\end{bmatrix}
\equiv
e^{100\pi i\theta}\begin{bmatrix}
1&0\\
0&e^{e^{i\theta}}
\end{bmatrix}
$$

</details>


#### Let's have some fun
The code below can be used to make a gif of a rotating bloch vector.

For this example we are spinning around the Y axis but you can play around with different circuits gates in the for-loop 

In [ ]:
# Plotting the gif may take a short while...

# We will save many plots to make a gif 
frames = []

QuCirc = QuantumCircuit(1)

# The number os frames in out gif
n_frames = 40

for n in range(n_frames):
    
    # Scale angle so we get one full rotation when it is applied once for each frame 
    theta = 2*np.pi/n_frames

    # append another gate to the circuit, gates from previous loops all still apply
    QuCirc.ry(theta,0) 


    plot = plot_bloch_multivector(QuCirc)


    # This will write our plot to the gif frames 
    buf = io.BytesIO()
    plot.savefig(buf, format='png')
    frames.append(Image.open(buf))



# Filename for our gif 
GIF_PATH = 'gif.gif'
# Save the gif
frames[0].save(GIF_PATH,
               save_all = True,
               #append_images = frames[1:] + frames[1:-2][::-1],
               append_images = frames[1:],
               duration = 100,
               loop=0)

# Display the gif
display(IPImage(data=open(GIF_PATH,'rb').read(), format='png'))

#### Challenge:
- You now learnt how to do arbitrary rotations. Try to apply an arbitrary set of rotations that traces out an interesting curve on the bloch sphere. 
- See if you can use frames and maybe a for loop to animate your traced curve as a gif
- **Bonus** can you parameterize your curve in a for loop?

**Tip**: If you want to remove all gates from a quantum circuit (maybe you need a 'clean slate' each loop) you can call `.clear()`.